In [2]:
from audiocraft.data.audio import audio_write
import IPython.display as ipd
from audiocraft.models import MusicGen, MultiBandDiffusion
import numpy as np

import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
# pip install torch==2.1.2 torchvision==0.16.2 torchaudio==2.1.2 --index-url https://download.pytorch.org/whl/cu118

# load your finetune
#musicgen = MusicGen.get_pretrained('./checkpoints/finetune')
musicgen = MusicGen.get_pretrained('facebook/musicgen-large')

musicgen.set_generation_params(duration=10)
mbd = MultiBandDiffusion.get_mbd_musicgen()

/home/nao/anaconda3/envs/audiocraft/lib/python3.9/site-packages/torch/nn/utils/weight_norm.py:30: UserWarning: torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.
  warnings.warn("torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.")


In [3]:
# Import required libraries
import openai
import json

# Initialize OpenAI API key
openai.api_key = ""  # Replace with your actual API key

In [4]:
# prompt_sample = f"""""I need text prompts for MusicGen, text-to-music model. I want to generate minimal techno, house, bass, garage music. Please give me 3 text promots and put in a python array. I like music with the combinations of two of the following characteristics, but please rephrase these words:
# groovy, bouncy, bassy, dubby, glitchy, jazzy, minimal, experimental, happy, tropical. BPM should be inbetween 110 to 135. Get creative and use random instruments! 
# """""

# # prompt_input = f"""""
# # Summerize '{item['text']}' and format the caption with other information into a JSON file.
# #"""""
# example_captions = [
#     "A 120 BPM upbeat techno track.",
#     "A 130 BPM smooth house music tune with a soothing saxophone solo, mellow piano chords, and soft percussion.",
#     "A heartwarming folk tune with soothing harmonies and gentle acoustic melodies, perfect for a relaxing afternoon in the countryside.",
# ]

# prompt_sample = f"""""I need text prompts for MusicGen, text-to-music model. I want to generate various percussion loops. Please give me 3 text promots and put in a python array. BPM should be inbetween 110 to 135. Get creative and use random instruments! 
# """""

# prompt_sample = f"""""I need text prompts for MusicGen, text-to-music model. I want to generate various melody loops than contain no drums nor percussions. Please give me 3 text promots and put in a python array. BPM should be inbetween 110 to 135. Be specific, get creative and use random instruments! 
# """""

prompt_sample = f"""""I need text prompts for MusicGen, text-to-music model. \
    Please give me 3 text promots and put in a python array. 
    It should be dance music such as house, techno, dubstep, juke, breakbeats, drum and bass, jungle, garage, etc. \ 
    I want make them experimental, but not too glitchy. It should be catchy, weird and pop and escpecially bassy. \
    please use lots of random instruments and effects. 
    BPM should be inbetween 110 to 180. Be specific, get creative and use random instruments! \
"""""


# prompt_input = f"""""
# Summerize '{item['text']}' and format the caption with other information into a JSON file.
#"""""
example_captions = [
    "African percussion drums with 808 bass and Shakuhachi, bpm 120",
    "Japanese taiko in jungle music with heavy bass in 2050, bpm 170",
    "A very groovy garage beat with a deep bassline and a catchy melody, bpm 140"
]


def get_captions():
    # Query
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "You are a world-famous dance music DJ."},
            {"role": "user", "content": f"{prompt_sample}"},
            {"role": "assistant", "content": f"{example_captions}"},
            {"role": "user", "content": f"{prompt_sample}"},
        ]
    )
    #print(response)

    # Extract caption from API response
    json_output = response['choices'][0]['message']['content'].strip()
    json_output = json_output.replace('\'', '\"')
    json_output = json.loads(json_output)
    print(json_output, type(json_output))
    return json_output

In [5]:
# # Example 1: unconditional generation
# #torch.manual_seed(12)

# wavs, tokens = musicgen.generate_unconditional(4, return_tokens=True)
# wav_diffusion = mbd.tokens_to_wav(tokens)
# #print(tokens.shape)

# # save and display generated audio
# for idx, one_wav in enumerate(wavs):
#     audio_write(f'{idx}', one_wav.cpu(), musicgen.sample_rate, strategy="loudness", loudness_compressor=True)
#     ipd.display(ipd.Audio(one_wav.cpu(), rate=32000))
#     audio_write(f'{idx}_diffusion', wav_diffusion[idx].cpu(), musicgen.sample_rate, strategy="loudness", loudness_compressor=True)

#     ipd.display(ipd.Audio(wav_diffusion[idx].detach().cpu(), rate=32000))

In [6]:
# Example 2: text guided generation
# import torch
# torch.manual_seed(12)

# wavs, tokens = musicgen.generate([
# 'Techno music, A midnight jazz club ambiance with minimal techno vibes.',
# 'House music, Deep basslines and dubby echoes in a warehouse setting.',
# 'House music, Serene beach sunset translated into minimal house melodies.',
# 'Techno music, Syncopated beats and smooth chord progressions, a jazzy touch.'
# ], return_tokens=True)
from IPython.display import clear_output

musicgen.set_generation_params(duration=14.0)

output_dir = 'generated_drums2'

for i in range(5000):
    try: 
        captions = get_captions()

        wavs, tokens = musicgen.generate(captions, return_tokens=True)
        wav_diffusion = mbd.tokens_to_wav(tokens)

        # save and display generated audio
        for idx, one_wav in enumerate(wavs):
            audio_write(f'{idx}', one_wav.cpu(), musicgen.sample_rate, strategy="loudness", loudness_compressor=True)
            ipd.display(ipd.Audio(one_wav.cpu(), rate=32000))
            caption = captions[idx]
            audio_write(f'{output_dir}/{i}_{caption}', wav_diffusion[idx].cpu(), musicgen.sample_rate, strategy="loudness", loudness_compressor=True)
            ipd.display(ipd.Audio(wav_diffusion[idx].detach().cpu(), rate=32000))
        clear_output(wait=True)
    except Exception as e:
        print(e)

['Futuristic techno groove with distorted trumpet solo and pulsating bassline, bpm 150', 'Dubstep wobble bass meets accordion melody in a dark alley, bpm 130', 'Experimental breakbeats with sitar riffs and laser sound effects, bpm 160'] <class 'list'>
